# Playing with the data structure

In [1]:
%load_ext autoreload
%autoreload 2

import malt
import torch
import seaborn as sns
from malt.molecule import Molecule

Using backend: pytorch


In [2]:
from pathlib import Path
f = f'{Path.home()}/dev/choderalab/data/data/moonshot_fluorescence_titration_curves.csv'
import pandas as pd
df = pd.read_csv(f, index_col=0).dropna()
df = df.rename({'concentration': 'c', 'inhibition': 'y'}, axis=1)

In [3]:
from dgllife.utils import smiles_to_bigraph, CanonicalAtomFeaturizer, CanonicalBondFeaturizer

def parse_graph(smiles):
    return smiles_to_bigraph(
        smiles = smiles,
        node_featurizer = CanonicalAtomFeaturizer(),
        edge_featurizer = CanonicalBondFeaturizer()
    )

In [11]:
from malt import Dataset, AssayedMolecule
from tqdm import tqdm

molecules = []
for smiles, mol_metadata in tqdm(df.groupby('SMILES')):
    molecule = AssayedMolecule(
        smiles = smiles,
        # g = parse_graph(smiles),
        metadata = {'fluorescence': mol_metadata.drop('SMILES', axis=1).to_dict('records')}
    )
    molecules.append(molecule)
    
# create dataset
data = Dataset(molecules)

100%|████████████████████████████████████████████████████████████████████████████| 2281/2281 [00:01<00:00, 1364.54it/s]


In [6]:
%load_ext autoreload
%autoreload 2
import malt
malt.data.utils.batch_metadata

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<function malt.data.utils.batch_metadata(molecule, key, **kwargs)>

In [40]:
import copy

copy.deepcopy(data[0])

Brc1ccc(CN2C(=O)C(=O)c3cccc(Br)c23)cc1

In [30]:
dl = data.view(
    batch_size=1,
    batch_metadata=malt.data.utils.batch_assay_metadata
)

In [45]:
data[0]['fluorescence'] = 0

TypeError: 'AssayedMolecule' object does not support item assignment

In [37]:
for d in dl:
    print(len(d[1]))
    break

24


In [ ]:
for idx in tqdm(range(len(data))):
    t = torch.Tensor([d[key] for d in data[idx]['fluorescence']])

100%|███████████████████████████████████████████████████████████████████████████| 2281/2281 [00:00<00:00, 81468.80it/s]
